# 目的

#每日收盤後開始爬股價

#可以視情況條種是否抓營收，大概都隔月的10 後營收才會公布

## 【資料源】
來自 _BaseInfo  _CrawBase 會先去取基本相關資料 (股票號碼...等)

## 【輸出】
股票相關推薦圖表/數據

In [ ]:
import pandas as pd

#  Update Base Info

In [ ]:
 
#%run _BaseInfoETF.ipynb
%run _CrawBase.ipynb
#%run _LineMsg.ipynb
%run _genHtml.ipynb

%run RunRealTimeStock.ipynb

In [ ]:
def MainStock(stock_number, start_month,end_time=datetime.now().strftime("%Y-%m-%d"),updateRevenue=False):  
    strmsg ='【stock_number】'+stock_number+'\n' +'【start_month】'+start_month+'\n'+'【end_time】'+end_time+'\n' 
    try:
        #處理 Stock
        RowData_df_craw_stock,His_Stock ,isSuccess= craw_stock(stock_number,start_month,end_time)
        #data_final,df_final=DataCleaning(RowData_df_craw_stock)
        print("【craw_stock】 success")
        
        #處理 Revenue
        if(updateRevenue):
            RowData_df_craw_revenue,His_Stock=craw_revenue(stock_number)
            df_Revenue=RevenueDataCleaning(RowData_df_craw_revenue)
            print("【craw_revenue】 success")
            
        #處理相關指標轉成html 
        stock_data=data_process(RowData_df_craw_stock)
        
        level,interval_type, lower_bound, upper_bound=get_interval(stock_data.iloc[-1]['收盤價'],np.array(stock_data['收盤價']))
        stock_data['stock_number']=stock_number
        stock_data['level']=level
        stock_data['interval_type']=interval_type
        stock_data['lower_bound']=lower_bound
        stock_data['upper_bound']=upper_bound
        save_df_to_excel(stock_data)
        gen_html(stock_number,stock_data)

        #sendLineMessage('成功'+'\n'+strmsg )
    except:
        #sendLineMessage('失败'+'\n'+strmsg)
        print('MainStock Fail'+'\n'+strmsg)
        
    #return data_final,df_final,df_Revenue
    

# 清資料使用
type_='Stock_His'
strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
df=get_DB()
df[~((df['status'] != 'OK')& (df['status'] != '很抱歉，沒有符合條件的資料!'))].to_csv('BaseInfo/His_Stock.csv',index=False) 
DDD=df[((df['status'] != 'OK')& (df['status'] != '很抱歉，沒有符合條件的資料!'))]['stock_number'].unique()

In [ ]:
start_month='2023-01-01'  #起始月份
want_update_Stock=True
want_update_Revenue=False
file_path = 'D:\Project\Jupyter\Stock\Main\DailyResultLast.xlsx'   

for code in _baseInfo.dropna()['公司代號']:#.iloc[::-1]:
#for code in DDD:
    stock_number=str(code)
    if (want_update_Stock):        
        type_='Stock_His'
        strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
        delete_DB_stock_by_Month_Stock_number(strMonth,stock_number,type_)
    try:
        if(want_update_Revenue):
            MainStock(stock_number,start_month,want_update_Revenue)
        else:
            MainStock(stock_number,start_month)
    except Exception as error:
        print("An exception occurred:", error)
        print('Action Fail'+'\n')
        
    print(stock_number,'=====================>Data Ready')
   